In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from itertools import product
import numpy as np
import pandas as pd
import warnings
from joblib import Parallel, delayed

warnings.filterwarnings("ignore")

In [ ]:
from joblib import Parallel, delayed

def entrenar_y_evaluar_modelo(params, serie_temp, METRICA_EVAL):
    p, q, P, Q = params
    # Código para entrenar el modelo SARIMA y calcular métricas
    try:
        model = SARIMAX(serie_temp[:int(len(serie_temp)*0.8)],
                        order=(p, 0, q),
                        seasonal_order=(P, 0, Q, 376),
                        enforce_stationarity=False,
                        enforce_invertibility=False)
        fitted_model = model.fit(disp=False)
        pred = fitted_model.forecast(steps=len(serie_temp[int(len(serie_temp)*0.8):]))
        # Métricas de evaluación
        rmse = np.sqrt(mean_squared_error(serie_temp[int(len(serie_temp)*0.8):], pred))
        return (p, q, P, Q, rmse)
    except Exception as e:
        return (p, q, P, Q, str(e))

param_combinations = list(product([1,2,3], [1,2,3], [1], [1,2]))

# Paralelización en Colab Pro
resultados = Parallel(n_jobs=-1)(delayed(entrenar_y_evaluar_modelo)(params, serie_O, 'RMSE') for params in param_combinations)

# Mostrar resultados
print(resultados)

## Import Data

In [2]:
df_oferta = pd.read_csv('../data/datos_Oferta_Renovables_EU27_GWh.csv', index_col=0, parse_dates=True)
# establecer frecuencia diaria
df_oferta = df_oferta.asfreq('D')

# tomar la serie de datos acumuados en EU27
serie_O = df_oferta['EU27'].copy()
serie_O.index

DatetimeIndex(['1980-01-01', '1980-01-02', '1980-01-03', '1980-01-04',
               '1980-01-05', '1980-01-06', '1980-01-07', '1980-01-08',
               '1980-01-09', '1980-01-10',
               ...
               '2024-12-22', '2024-12-23', '2024-12-24', '2024-12-25',
               '2024-12-26', '2024-12-27', '2024-12-28', '2024-12-29',
               '2024-12-30', '2024-12-31'],
              dtype='datetime64[ns]', name='Date', length=16437, freq='D')

## Modelos SARIMA(p,0,q)(P,0,Q)m

In [3]:
def obtener_df_modelos_SARIMA(serie_temp, p_values, q_values, d, P_values, Q_values, D, m, METRICA_EVAL):
    '''
    Función para entrenar y evaluar modelos ARIMA en una serie temporal.
    Parametros:
    - serie_temp: Serie temporal a evaluar (pd.Series).
    - p_values: Lista de valores para el parámetro p (int).
    - q_values: Lista de valores para el parámetro q (int).
    - d: Diferenciación (int).
    - P_values: Lista de valores para el parámetro P (int).
    - Q_values: Lista de valores para el parámetro Q (int).
    - D: Diferenciación estacional (int).
    - m: Periodo estacional (int).
    - METRICA_EVAL: Métrica de evaluación (str) para ordenar los resultados ('AIC', 'BIC', 'RMSE', 'MAPE', 'R2').
    '''
    
    dict_metricas_eval = {
        'AIC': True,  # AIC menor es mejor (ascending=True)
        'BIC': True,  # BIC menor es mejor
        'RMSE': True,  # RMSE menor es mejor
        'MAPE': True,  # MAPE menor es mejor
        'R2': False  # R2 mayor es mejor (ascending=False)
    }

    # Train-test split
    train_size = int(len(serie_temp) * 0.8)
    train, test = serie_temp[:train_size], serie_temp[train_size:]

    # Crear combinaciones de parámetros
    param_combinations = list(product(p_values, q_values, P_values, Q_values)) 

    # Resultados
    results = []

    # Loop por combinaciones
    for i, (p,q,P,Q) in enumerate(param_combinations):
        try:
            print()
            print(f'Entrenando modelo SARIMA({p},{d},{q})({P},{D},{Q})[{m}]...')
            model = SARIMAX(train,
                            order=(p, d, q),
                            seasonal_order=(P, D, Q, m),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
            fitted_model = model.fit(disp=False)  

            # Forecast
            print('Prediciendo...')
            pred = fitted_model.forecast(steps=len(test))

            # Evaluación
            print('Evaluando...')
            aic = fitted_model.aic
            bic = fitted_model.bic
            rmse = np.sqrt(mean_squared_error(test, pred))
            mape = mean_absolute_percentage_error(test, pred)
            r2 = r2_score(test, pred)

            results.append({
                'p': p, 'q': q, 'P': P, 'Q': Q,
                'AIC': aic,
                'BIC': bic,
                'RMSE': rmse,
                'MAPE': mape,
                'R2': r2
            })

            print(f'Modelo {i+1}/{len(param_combinations)} listo.')

        except Exception as e:
            print(f'Error en modelo SARIMA({p},{d},{q})({P},{D},{Q})[{m}]: {e} de tipo {type(e).__name__}')
            continue

    # Convertir a DataFrame y ordenar por AIC (cuanto menor, mejor)
    df_arima = pd.DataFrame(results).sort_values(f'{METRICA_EVAL}', ascending=dict_metricas_eval[METRICA_EVAL])
    return df_arima

### SARIMA parámetros significativos
parámetros teóricamente más significativos:
- ```d=0``` :  serie estacionaria según test ADF y KPSS
- ```p``` : por el PACF vemos que los lags más significativos son ```[1,2,3]```.
- ```q``` : por el ACF vemos que los lags más significativos son ```[1,2,3]```.
- ```D=0```
- ```P``` : parece que es ```1```.
- ```Q``` : parece que puede tener algo más de estacionalidad a largo plazo, por lo que se puede probar con ```[1,2]```.
- ```m=376``` : estacionalidad anual, estimación según los datos.

#### serie parcial 2019-2024

In [4]:
serie_O_ej_01 = serie_O['2019':'2024']

df_sarima_01 = obtener_df_modelos_SARIMA(serie_O_ej_01, 
                                        p_values=[1,2,3], 
                                        q_values=[1,2,3], 
                                        d=0, 
                                        P_values=[1], 
                                        Q_values=[1,2], 
                                        D=0, 
                                        m=376, 
                                        METRICA_EVAL='R2')
df_sarima_01


Entrenando modelo SARIMA(1,0,1)(1,0,1)[376]...


KeyboardInterrupt: 

In [ ]:
# ordenando por AIC
df_sarima_01.sort_values('AIC', ascending=True)

,p,q,AIC,BIC,RMSE,MAPE,R2
5,2,3,25040.380713,25073.181510,593.624356,0.204867,-0.338058
8,3,3,25043.416035,25081.683631,593.587394,0.205245,-0.337892
7,3,2,25062.145581,25094.949808,591.784961,0.204952,-0.329779
2,1,3,25071.870284,25099.204282,575.532696,0.199100,-0.257742
6,3,1,25072.913051,25100.249906,586.699633,0.203613,-0.307023
4,2,2,25073.648552,25100.985408,594.022280,0.206166,-0.339853
3,2,1,25146.826934,25168.698703,597.206207,0.206689,-0.354255
1,1,2,25184.388325,25206.257809,842.804455,0.331990,-1.697151
0,1,1,25372.538987,25388.942814,1788.555351,0.852370,-11.146648


#### datos completos 1980-2024


In [ ]:
# parámetros significativos
df_sarima_teoria = obtener_df_modelos_SARIMA(serie_O, 
                                        p_values=[1,2,3], 
                                        q_values=[1,2,3], 
                                        d=0, 
                                        P_values=[1], 
                                        Q_values=[1,2], 
                                        D=0, 
                                        m=376, 
                                        METRICA_EVAL='R2')
df_sarima_teoria


Entrenando modelo ARIMA(1,0,1)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 1/9 listo.

Entrenando modelo ARIMA(1,0,2)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 2/9 listo.

Entrenando modelo ARIMA(1,0,3)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 3/9 listo.

Entrenando modelo ARIMA(2,0,1)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 4/9 listo.

Entrenando modelo ARIMA(2,0,2)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 5/9 listo.

Entrenando modelo ARIMA(2,0,3)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 6/9 listo.

Entrenando modelo ARIMA(3,0,1)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 7/9 listo.

Entrenando modelo ARIMA(3,0,2)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 8/9 listo.

Entrenando modelo ARIMA(3,0,3)(0,0,0)[0]...
Prediciendo...
Evaluando...
Modelo 9/9 listo.


,p,q,AIC,BIC,RMSE,MAPE,R2
7,3,2,188756.443206,188801.346443,606.326372,0.218216,-0.604054
8,3,3,188716.911076,188769.297653,612.418354,0.220630,-0.636449
5,2,3,188720.846905,188765.749686,614.617859,0.221398,-0.648224
4,2,2,188767.430035,188804.849399,639.350328,0.231507,-0.783544
6,3,1,188878.933381,188916.352745,643.588527,0.233247,-0.807268
2,1,3,188954.809235,188992.228218,897.641435,0.358100,-2.515699
1,1,2,189712.929685,189742.865176,1750.672880,0.853603,-12.372603
3,2,1,191134.043722,191163.979517,1926.955690,0.983892,-15.201283
0,1,1,191170.562870,191193.014716,1928.381747,0.984771,-15.225272


In [ ]:
df_sarima_teoria.to_csv('resultados_modelos/SARIMA_parametros_significativos_R2.csv', index=False)

In [ ]:
df_sarima_teoria_AIC = df_sarima_teoria.sort_values('AIC', ascending=True)

,p,q,AIC,BIC,RMSE,MAPE,R2
8,3,3,188716.911076,188769.297653,612.418354,0.220630,-0.636449
5,2,3,188720.846905,188765.749686,614.617859,0.221398,-0.648224
7,3,2,188756.443206,188801.346443,606.326372,0.218216,-0.604054
4,2,2,188767.430035,188804.849399,639.350328,0.231507,-0.783544
6,3,1,188878.933381,188916.352745,643.588527,0.233247,-0.807268
2,1,3,188954.809235,188992.228218,897.641435,0.358100,-2.515699
1,1,2,189712.929685,189742.865176,1750.672880,0.853603,-12.372603
3,2,1,191134.043722,191163.979517,1926.955690,0.983892,-15.201283
0,1,1,191170.562870,191193.014716,1928.381747,0.984771,-15.225272


In [ ]:
df_sarima_teoria_AIC.to_csv('resultados_modelos/SARIMA_parametros_significativos_AIC.csv', index=False)